In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
# Create DataFrame
import pandas as pd

df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])


In [9]:
df.tail()

,course,section,question,text
943,mlops-zoomcamp,Module 6: Best practices,Github actions: Permission denied error when e...,Problem description\nThis is the step in the c...
944,mlops-zoomcamp,Module 6: Best practices,Managing Multiple Docker Containers with docke...,Problem description\nWhen a docker-compose fil...
945,mlops-zoomcamp,Module 6: Best practices,AWS regions need to match docker-compose,Problem description\nIf you are having problem...
946,mlops-zoomcamp,Module 6: Best practices,Isort Pre-commit,Problem description\nPre-commit command was fa...
947,mlops-zoomcamp,Module 6: Best practices,How to destroy infrastructure created via GitH...,Problem description\nInfrastructure created in...


In [11]:
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [13]:
documents = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(documents)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
15th,1,0,0,0,0
2024,1,0,0,0,0
cloud,0,0,0,0,1
course,1,0,0,0,1
date,0,0,1,0,0
github,0,1,0,0,0
google,0,0,0,0,1
homeworks,0,0,1,0,0
jan,1,0,0,0,0
listed,0,1,0,0,0


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(documents)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4
15th,0.46,0.00,0.0,0.00,0.00
2024,0.46,0.00,0.0,0.00,0.00
cloud,0.00,0.00,0.0,0.00,0.46
course,0.37,0.00,0.0,0.00,0.37
date,0.00,0.00,0.5,0.00,0.00
github,0.00,0.58,0.0,0.00,0.00
google,0.00,0.00,0.0,0.00,0.46
homeworks,0.00,0.00,0.5,0.00,0.00
jan,0.46,0.00,0.0,0.00,0.00
listed,0.00,0.58,0.0,0.00,0.00


In [20]:
query = "Do I need to know python to sign up for the January course?"

q = cv.transform([query])
q.toarray()

array([[0.        , 0.        , 0.        , 0.62791376, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.77828292, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [24]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict



{'15th': np.float64(0.0),
 '2024': np.float64(0.0),
 'cloud': np.float64(0.0),
 'course': np.float64(0.6279137616509933),
 'date': np.float64(0.0),
 'github': np.float64(0.0),
 'google': np.float64(0.0),
 'homeworks': np.float64(0.0),
 'jan': np.float64(0.0),
 'listed': np.float64(0.0),
 'participation': np.float64(0.0),
 'prerequisites': np.float64(0.0),
 'python': np.float64(0.7782829228046183),
 'registration': np.float64(0.0),
 'required': np.float64(0.0),
 'setup': np.float64(0.0),
 'start': np.float64(0.0),
 'starts': np.float64(0.0),
 'submit': np.float64(0.0)}

In [23]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

{'15th': np.float64(0.0),
 '2024': np.float64(0.0),
 'cloud': np.float64(0.0),
 'course': np.float64(0.0),
 'date': np.float64(0.0),
 'github': np.float64(0.5773502691896258),
 'google': np.float64(0.0),
 'homeworks': np.float64(0.0),
 'jan': np.float64(0.0),
 'listed': np.float64(0.5773502691896258),
 'participation': np.float64(0.0),
 'prerequisites': np.float64(0.5773502691896258),
 'python': np.float64(0.0),
 'registration': np.float64(0.0),
 'required': np.float64(0.0),
 'setup': np.float64(0.0),
 'start': np.float64(0.0),
 'starts': np.float64(0.0),
 'submit': np.float64(0.0)}

In [25]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

(df_qd['query'] * df_qd['doc']).sum()

np.float64(0.0)

In [26]:
X.dot(q.T).toarray()

array([[0.23490553],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59579005]])

In [29]:
cosine_similarity(X, q)

array([[0.23490553],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.59579005]])

In [30]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

transformers['text'].get_feature_names_out()
matrices['text']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 26463 stored elements and shape (948, 2118)>

In [31]:
query = "I just singned up. Is it too late to join the course?"

q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

In [32]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score

array([0.3336047 , 0.        , 0.        , 0.1328874 , 0.        ,
       0.        , 0.        , 0.12722114, 0.        , 0.        ,
       0.        , 0.10830554, 0.        , 0.        , 0.        ,
       0.23530268, 0.        , 0.        , 0.04595339, 0.        ,
       0.        , 0.        , 0.22668   , 0.07952931, 0.        ,
       0.        , 0.        , 0.1894954 , 0.08310739, 0.        ,
       0.        , 0.        , 0.        , 0.03724346, 0.        ,
       0.        , 0.        , 0.        , 0.16484429, 0.0231432 ,
       0.03424155, 0.05174621, 0.        , 0.03167699, 0.        ,
       0.        , 0.        , 0.        , 0.02097473, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04722243, 0.        , 0.0073737 ,
       0.        , 0.        , 0.        , 0.        , 0.04161211,
       0.        , 0.        , 0.        , 0.        , 0.     

In [34]:
import numpy as np

idx = np.argsort(-score)[:10]
idx

array([  0,  15,  22,  27,  38, 287,   3,   7, 113,  11])

In [35]:
df.iloc[idx].text

0      The purpose of this document is to capture fre...
15     No, late submissions are not allowed. But if t...
22     It's up to you which platform and environment ...
27     You can do most of the course without a cloud....
38     You will have two attempts for a project. If t...
287    This error could result if you are using some ...
3      You don't need it. You're accepted. You can al...
7      Yes, we will keep all the materials after the ...
113    In the join queries, if we mention the column ...
11     No, you can only get a certificate if you fini...
Name: text, dtype: object

In [36]:
boost = {'question': 3.0}

score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s